## Data pull, parsing and preliminary modeling
#### CS109a - Sam, Luke, Jake, Jeremy
#### Chicago Food Inspection Forecasting
#### Last Updated: 11/28/16

In [4]:
from download import main

In [5]:
import datetime
import optparse
import requests
import numpy as np
import pandas as pd
from math import radians, cos, sin, asin, sqrt
from sys import exit
import math
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
main(date=datetime.datetime.strptime('01/01/2015', '%m/%d/%Y'), export=True)

Fetching data beginning 2015-01-01.
38800 records received. Finding businesses...
Succeeded!
Succeeded!
Succeeded!
Succeeded!
Succeeded!
Succeeded!
Succeeded!
Succeeded!
Succeeded!
Succeeded!
Succeeded!
Succeeded!
Succeeded!
Succeeded!
Succeeded!
Succeeded!
Succeeded!
Succeeded!
Succeeded!
Succeeded!
Succeeded!
{u'errorCode': u'query.soql.type-mismatch', u'message': u'Query coordinator error: query.soql.type-mismatch; Type mismatch for #IN, is text; position: Map(row -> 1, column -> 765, line -> "SELECT * WHERE `license_number` IN (\'1766122\',\'55059\',\'2093205\',\'3305802\',\'2327216\',\'55054\',\'2405981\',\'51574\',\'2333015\',\'2443184\',\'1197155\',\'2138566\',\'1677809\',\'2354588\',\'66710\',\'1878490\',\'2354580\',\'2442851\',\'2442853\',\'1766349\',\'2294041\',\'1069050\',\'2142330\',\'1739804\',\'1717915\',\'1717917\',\'2404720\',\'2308785\',\'1094390\',\'12532\',\'2446795\',\'2404721\',\'2233259\',\'2327082\',\'2443459\',\'1334960\',\'2036917\',\'2215771\',\'2215770\',\'22

/usr/local/lib/python2.7/site-packages/numpy/lib/arraysetops.py:200: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))
Exception 

KeyboardInterrupt in 'zmq.backend.cython.message.Frame.__dealloc__' ignored


KeyboardInterrupt: 

First we load in the data:

In [ ]:
data = pd.read_csv('DOWNLOADED_DATA_1.csv')

Now we check out the columns:

These functions build binary indicators for license types 

In [ ]:
def fail(result):
    if 'Fail' in result:
        return 0
    else:
        return 1
    
def tobacco(license_description):
    if 'Tobacco Retail Over Counter' == license_description:
        return 1
    else:
        return 0
    
def liquor(license_description):
    if 'Consumption on Premises - Incidental Activity' == license_description:
        return 1
    else:
        return 0
    
def grocery_store(facility_type):
    if 'Grocery Store' == facility_type:
        return 1
    else:
        return 0

In [ ]:
data['result_binary'] = [fail(result) for result in data.results]
data['consumption_license'] = [liquor(license) for license in data.license_description]
data['tobacco'] = [tobacco(license) for license in data.license_description]
data['grocery'] = [grocery_store(facility_type) for facility_type in data.facility_type]

In [ ]:
data['inspection_date'] = pd.to_datetime(data['inspection_date'])
data['license_start_date'] = pd.to_datetime(data['license_start_date'])
data['date_issued'] = pd.to_datetime(data['date_issued'])
data['creation_date'] = pd.to_datetime(data['creation_date'])

In [ ]:
import re

critical_flags = re.compile(r'(\bCRITICAL\b)',flags = re.IGNORECASE)
serious_flags = re.compile(r'(\bSERIOUS\b)',flags = re.IGNORECASE)
citation_flags = re.compile(r'(\bCITATION\b)',flags = re.IGNORECASE)
violation_flags = re.compile(r'(\bVIOLATION\b)',flags = re.IGNORECASE)
minor_flags = re.compile(r'(\bMINOR\b)',flags = re.IGNORECASE)


In [ ]:
violations_count = []
citations_count = []
critical_count = []
serious_count = []
minor_count = []

LOG_EVERY_N = 10000


  
for index in data.index:
    text = data.ix[index].violations
    
    if type(text) == type(' '):
        violations_count.append(violation_flags.subn('',text)[1])
        citations_count.append(citation_flags.subn('',text)[1])
        critical_count.append(critical_flags.subn('',text)[1])
        serious_count.append(serious_flags.subn('',text)[1])
        minor_count.append(minor_flags.subn('',text)[1])

    else:
        violations_count.append(float('nan'))
        citations_count.append(float('nan'))
        critical_count.append(float('nan'))
        serious_count.append(float('nan'))
        minor_count.append(float('nan'))
        
    if (index % LOG_EVERY_N) == 0:
        print index


In [ ]:
data['violations_count'] = violations_count
data['citations_count'] = citations_count
data['critical_count'] = critical_count
data['serious_count'] = serious_count
data['minor_count'] = minor_count



In [ ]:
# compare pass rates for different violation counts
bins = np.linspace(0,6,7)
data[data.result_binary == 1].citations_count.hist(normed=False,alpha=0.5,bins=bins)
data[data.result_binary == 0].citations_count.hist(normed=False,alpha=0.5,bins=bins)

In [ ]:
def prev_inspection_features(data,inspection_id,date):
    current_inspection = data.ix[data[data.inspection_id==inspection_id].index[0]]
    
    all_inspections = data[data.license_ == current_inspection.license_]
    
    previous_inspections = all_inspections[all_inspections.inspection_date < date]
    if len(previous_inspections) >= 1:
        last_inspection_index = previous_inspections.inspection_date.sort_values().index[-1]
        fraction = previous_inspections.result_binary.mean()
        count = previous_inspections.result_binary.count()
        last = previous_inspections.result_binary[last_inspection_index]
        
        if type(pd.to_datetime('nan')) != type(current_inspection.license_start_date):
            license_age = current_inspection.inspection_date - current_inspection.license_start_date
            license_age = license_age.days
            if license_age <= 0:
                license_age = 0
        else:
            license_age = float('nan')
            
        time_since_last = current_inspection.inspection_date - previous_inspections.ix[last_inspection_index].inspection_date
        time_since_last = time_since_last.days
        
        prev_violations = previous_inspections.violations_count[last_inspection_index]
        prev_citations = previous_inspections.citations_count[last_inspection_index]
        prev_critical = previous_inspections.critical_count[last_inspection_index]
        prev_serious = previous_inspections.serious_count[last_inspection_index]
        prev_minor = previous_inspections.minor_count[last_inspection_index]
        
        
        
    else:
        fraction = float('nan')
        count = 0
        last = float('nan')
        license_age = float('nan')
        time_since_last = float('nan')
        prev_violations = float('nan')
        prev_citations = float('nan')
        prev_critical = float('nan')
        prev_serious = float('nan')
        prev_minor = float('nan')
        
        
        
        
    return np.array([count,
                     fraction,
                     last,
                     license_age,
                     time_since_last,
                     prev_violations,
                     prev_citations,
                     prev_critical,
                     prev_serious,
                     prev_minor])
    
    


In [ ]:
previous_inspection_features = np.zeros((len(data.index),10))


LOG_EVERY_N = 10000
for index in data.index:
    previous_inspection_features[index,:] = prev_inspection_features(data,data.ix[index].inspection_id,data.ix[index].inspection_date)
    if (index % LOG_EVERY_N) == 0:
        print index


In [ ]:
data['previous_count'] = previous_inspection_features[:,0]
data['previous_fraction'] = previous_inspection_features[:,1]
data['previous_result'] = previous_inspection_features[:,2]
data['license_age'] = previous_inspection_features[:,3]
data['time_since_last_inspection'] = previous_inspection_features[:,4]
data['previous_violations'] = previous_inspection_features[:,5]
data['previous_citations'] = previous_inspection_features[:,6]
data['previous_critical'] = previous_inspection_features[:,7]
data['previous_serious'] = previous_inspection_features[:,8]
data['previous_minor'] = previous_inspection_features[:,9]



In [ ]:
for col in data.columns:
    print col

In [ ]:

#adjust number of buckets using np.linspace ----------------->
yedges = np.linspace(data.longitude.min(),data.longitude.max(),6)
xedges = np.linspace(data.latitude.min(),data.latitude.max(),6)

x = np.array(data.latitude)
y = np.array(data.longitude)

x1 = np.array(data[data.result_binary == 1].latitude)
y1 = np.array(data[data.result_binary == 1].longitude)

x0 = np.array(data[data.result_binary == 0].latitude)
y0 = np.array(data[data.result_binary == 0].longitude)


hist0, xedges, yedges = np.histogram2d(x0, y0, (xedges, yedges))
hist1, xedges, yedges = np.histogram2d(x1, y1, (xedges, yedges))

hist = hist0/(hist1+hist0)
hist_count = hist1+hist0

xidx = np.clip(np.digitize(x, xedges), 0, hist.shape[0]-1)
yidx = np.clip(np.digitize(y, yedges), 0, hist.shape[1]-1)

c = hist[xidx, yidx]
plt.scatter(x, y, c=c)

plt.show()

sorted_index = np.argsort(hist.reshape(1,-1))

print hist_count.reshape(1,-1)[0][sorted_index[0]]
print hist.reshape(1,-1)[0][sorted_index[0]]

In [ ]:
data['bin_index'] = hist.shape[1]*yidx + xidx

fig = plt.figure(figsize=(10,10)) # Create matplotlib figure

ax = fig.add_subplot(111) # Create matplotlib axes
ax2 = ax.twinx() # Create another axes that shares the same x-axis as ax.

width = 0.4


data.groupby('bin_index').count().result_binary.plot(kind='bar', color='red', ax=ax, width=width, position=1)
data.groupby('bin_index').mean().result_binary.plot(kind='bar', color='blue', ax=ax2, width=width, position=0)


plt.show()

In [ ]:
result_index = data.groupby('license_description').mean().result_binary.sort_values().index
data.groupby('license_description').mean().result_binary.sort_values()[result_index]

In [ ]:

fig = plt.figure(figsize=(10,10)) # Create matplotlib figure

ax = fig.add_subplot(111) # Create matplotlib axes
ax2 = ax.twinx() # Create another axes that shares the same x-axis as ax.

width = 0.4


data.groupby('license_description').count().result_binary[result_index].plot(kind='bar', color='red', ax=ax, width=width, position=1)
data.groupby('license_description').mean().result_binary.sort_values()[result_index].plot(kind='bar', color='blue', ax=ax2, width=width, position=0)


plt.show()



In [ ]:
top_50 = data.groupby('facility_type').count().result_binary.sort_values().index[-50:]

In [ ]:
data.groupby('facility_type').mean().result_binary[top_50].sort_values()

top_50count_mean_sorted_index = data.groupby('facility_type').mean().result_binary[top_50].sort_values().index



In [ ]:

fig = plt.figure(figsize=(10,10)) # Create matplotlib figure

ax = fig.add_subplot(111) # Create matplotlib axes
ax2 = ax.twinx() # Create another axes that shares the same x-axis as ax.

width = 0.4


data.groupby('facility_type').mean().result_binary[top_50].sort_values().plot(kind='bar', color='red', ax=ax, width=width, position=1)
data.groupby('facility_type').count().result_binary[top_50count_mean_sorted_index].plot(kind='bar', color='blue', ax=ax2, width=width, position=0)


plt.show()


#### Baseline modeling

In [ ]:
# Simple averaging

simple_avg_classifier = []

for frac in data.previous_fraction:
    if frac > 0.5:
        simple_avg_classifier.append(1)
    elif frac < 0.5:
        simple_avg_classifier.append(1)
    
    else:
        simple_avg_classifier.append(np.random.choice([0,1]))
        
print np.mean(np.array(simple_avg_classifier) == data.result_binary)

In [ ]:
cols = ['results', 
        'result_binary', 
        'risk', 
        'sanit_count', 
        'crime_count', 
        'consumption_license', 
        'tobacco', 
        'grocery', 
        'previous_count',
        'minor_count',
        'previous_fraction', 
        'previous_result', 
        'license_age', 
        'time_since_last_inspection', 
        'previous_violations', 
        'previous_citations', 
        'previous_critical', 
        'previous_serious', 
        'previous_minor', 
        'bin_index']

print data[cols].shape
print data.shape

# select specific features
data_sub = data[cols]
# drop NaN
data_sub = data_sub.dropna(axis=0)

In [ ]:
# check response
print 'Useless response count:', sum((data_sub.results != 'Pass') & (data_sub.results != 'Pass w/ Conditions') & (data_sub.results != 'Fail'))
indices = (data_sub.results == 'Pass') | (data_sub.results == 'Pass w/ Conditions') | (data_sub.results == 'Fail')
data_sub = data_sub[indices]

In [ ]:
# response
y2 = data_sub.result_binary
y3 = data_sub.results
# features
x = data_sub.drop(['result_binary', 'results'], axis = 1) 

# print data_sub.shape, data_sub_x.shape, data_sub_y_2, data_sub_y_3

In [ ]:
# encode
x_encoded = pd.DataFrame({})
for column in x.columns:
    if (x[column].unique().shape[0] < 8) or x[column].dtype == np.dtype('object'):
        encoding = pd.get_dummies(x[column])
        x_encoded = pd.concat([x_encoded, encoding], axis = 1)
    else:
        x_encoded = pd.concat([x_encoded, x[[column]].astype(float)], axis = 1)

In [ ]:
from sklearn import linear_model

baseline = linear_model.LogisticRegression()
baseline.fit(x_encoded, y2)
print baseline.score(x_encoded, y2)

baseline = linear_model.LogisticRegression(solver='newton-cg', multi_class = 'multinomial')
baseline.fit(x_encoded, y3)
print baseline.score(x_encoded, y3)

In [ ]:
# arbitary split
from sklearn.cross_validation import train_test_split as sk_split
test_size = 0.20
x_train, x_test, y2_train, y2_test = sk_split(x_encoded, y2, test_size = test_size)

baseline = linear_model.LogisticRegression()
baseline.fit(x_train, y2_train)
print 'Score:', baseline.score(x_test, y2_test)
print 'Score for passing inspections:', baseline.score(x_train[y2_train == 1], y2_train[y2_train == 1])
print 'Score for non-passing inspections:', baseline.score(x_train[y2_train == 0], y2_train[y2_train == 0])

In [ ]:
# class counts in training set

print 'Count did not pass:', sum(y2_train == 0)
print 'Count did pass:', sum(y2_train == 1)

In [ ]:
# quick fix for imbalance

# takes x_train, y_train and returns a class-balanced x_train, y_train 
# assumes count of class 0 > count of class 1
def subsample(x_train, y_train):
    train = pd.concat([x_train, y_train], axis = 1)

    train_0 = train[y_train == 0]
    train_1 = train[y_train == 1]
    
    train_1_subsample = train_1.sample(train_0.shape[0])
    
    train_subsample = pd.concat([train_1_subsample, train_0], axis = 0)
    
    x_train_subsample = train_subsample.iloc[:, :-1]
    y_train_subsample = train_subsample.iloc[:, -1]
    
    return x_train_subsample, y_train_subsample
    
## test subsampling algorithm with same simple classifier
x_train_sub, y2_train_sub = subsample(x_train, y2_train)

print x_train_sub.shape
print y2_train_sub.shape


In [ ]:
baseline_cls_corr = linear_model.LogisticRegression()
baseline_cls_corr.fit(x_train_sub, y2_train_sub)
print 'Score:', baseline_cls_corr.score(x_test, y2_test)
print 'Score for passing inspections:',baseline_cls_corr.score(x_test[y2_test == 1], y2_test[y2_test == 1])
print 'Score for non-passing inspections:', baseline_cls_corr.score(x_test[y2_test == 0], y2_test[y2_test == 0])